In [1]:
%%configure -f
{
    "conf": {
        "spark.app.name": "datavirus_timetable"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7587,application_1589299642358_2082,pyspark,idle,Link,Link,
7598,application_1589299642358_2093,pyspark,idle,Link,Link,
7604,application_1589299642358_2099,pyspark,idle,Link,Link,
7606,application_1589299642358_2101,pyspark,idle,Link,Link,
7608,application_1589299642358_2103,pyspark,busy,Link,Link,
7611,application_1589299642358_2106,pyspark,idle,Link,Link,
7616,application_1589299642358_2111,pyspark,idle,Link,Link,
7617,application_1589299642358_2112,pyspark,idle,Link,Link,
7618,application_1589299642358_2113,pyspark,busy,Link,Link,
7619,application_1589299642358_2114,pyspark,idle,Link,Link,


In [2]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7640,application_1589299642358_2135,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7587,application_1589299642358_2082,pyspark,idle,Link,Link,
7598,application_1589299642358_2093,pyspark,idle,Link,Link,
7604,application_1589299642358_2099,pyspark,idle,Link,Link,
7606,application_1589299642358_2101,pyspark,idle,Link,Link,
7608,application_1589299642358_2103,pyspark,busy,Link,Link,
7611,application_1589299642358_2106,pyspark,idle,Link,Link,
7616,application_1589299642358_2111,pyspark,idle,Link,Link,
7617,application_1589299642358_2112,pyspark,idle,Link,Link,
7618,application_1589299642358_2113,pyspark,busy,Link,Link,
7619,application_1589299642358_2114,pyspark,idle,Link,Link,


In [4]:
import pyspark.sql.functions as f

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
stop_time = spark.read.orc("/data/sbb/timetables/orc/stop_times/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
stop_time.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- trip_id: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- stop_id: string (nullable = true)
 |-- stop_sequence: short (nullable = true)
 |-- pickup_type: byte (nullable = true)
 |-- drop_off_type: byte (nullable = true)

In [7]:
stop_time.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+-------+-------------+-----------+-------------+
|             trip_id|arrival_time|departure_time|stop_id|stop_sequence|pickup_type|drop_off_type|
+--------------------+------------+--------------+-------+-------------+-----------+-------------+
|1119.TA.5-8-j19-1...|    16:26:00|      16:26:00|8589347|            3|          0|            0|
|65.TA.16-733-j19-...|    13:08:00|      13:08:00|8578794|            5|          0|            0|
|127.TA.16-300-j19...|    07:48:00|      07:48:00|8574314|           24|          0|            0|
|589.TA.26-225-j19...|    12:06:00|      12:06:00|8573733|            7|          0|            0|
|455.TA.5-15-j19-1...|    12:44:00|      12:44:00|8500147|            8|          0|            0|
|126.TA.1-444-j19-...|    11:06:00|      11:06:00|8580847|            2|          0|            0|
|1141.TA.26-46-j19...|    15:41:00|      15:41:00|8591358|            5|          0|            0|
|1289.TA.6

In [8]:
@f.udf
def parent_station_id(stop_id):
    return stop_id.split(':')[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
stop_time = (
    stop_time
    .select(['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence'])
    .withColumn('stop_id', parent_station_id(stop_time.stop_id))
    .filter(
        (stop_time.departure_time >= '05:00:00')
        & (stop_time.departure_time <= '21:00:00')
        & (stop_time.arrival_time >= '05:00:00')
        & (stop_time.arrival_time <= '21:00:00')
    )
    
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
(
    stop_time
    .groupBy(['trip_id', 'arrival_time'])
    .agg(
        f.collect_list(f.col('stop_sequence')).alias('sequence'), 
        f.collect_list(f.col('stop_id')).alias('stops'), 
        f.countDistinct(f.col('stop_id')).alias('count'),
    )
    .where(f.col('count') > 5)
).show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+------------+------------------------------------+---------------------------------------------------------------------------------+-----+
|trip_id                |arrival_time|sequence                            |stops                                                                            |count|
+-----------------------+------------+------------------------------------+---------------------------------------------------------------------------------+-----+
|54.TA.23-15-A-j19-1.3.R|07:58:00    |[7, 8, 6, 9, 10, 11]                |[8504445, 8504446, 8504444, 8504447, 8504433, 8504437]                           |6    |
|16.TA.23-15-A-j19-1.1.H|20:29:00    |[3, 4, 5, 6, 7, 2]                  |[8581046, 8581047, 8581048, 8505195, 8504407, 8593763]                           |6    |
|22.TA.23-15-A-j19-1.1.H|07:38:00    |[16, 17, 20, 15, 19, 18]            |[8504427, 8504428, 8504431, 8504426, 8504430, 8504429]                           |6    |
|46.TA.23-15-A-j

In [69]:
stop_time.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+-------+-------------+
|             trip_id|arrival_time|departure_time|stop_id|stop_sequence|
+--------------------+------------+--------------+-------+-------------+
|2462.TA.26-6-B-j1...|    16:31:00|      16:31:00|8588078|           10|
|1149.TA.26-200-j1...|    16:01:00|      16:02:00|8573726|            5|
|436.TA.26-235-j19...|    13:15:00|      13:15:00|8503694|            9|
|45.TA.1-322-j19-1...|    09:41:00|      09:41:00|8572564|           11|
|631.TA.6-2-A-j19-...|    06:08:00|      06:08:00|8504103|           19|
|303.TA.5-38-j19-1...|    18:48:00|      18:48:00|8588773|            4|
|1356.TA.26-215-j1...|    11:18:00|      11:18:00|8573720|           16|
|1002.TA.26-223-j1...|    14:32:00|      14:32:00|8502758|            1|
|148.TA.26-650-j19...|    20:24:00|      20:24:00|8575939|            7|
|163.TA.26-122-j19...|    06:20:00|      06:20:00|8590500|            6|
+--------------------+------------+--------------+-

In [70]:
stations = spark.read.csv("../data/zurich_stations_ids.csv")
stations = (
    stations
    .select(stations._c0.alias('stop_id'))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
filtered = (
    stop_time
    .join(f.broadcast(stations), 'stop_id')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
filtered.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+------------+--------------+-------------+
|stop_id|             trip_id|arrival_time|departure_time|stop_sequence|
+-------+--------------------+------------+--------------+-------------+
|8588078|2462.TA.26-6-B-j1...|    16:31:00|      16:31:00|           10|
|8573726|1149.TA.26-200-j1...|    16:01:00|      16:02:00|            5|
|8503694|436.TA.26-235-j19...|    13:15:00|      13:15:00|            9|
|8573720|1356.TA.26-215-j1...|    11:18:00|      11:18:00|           16|
|8502758|1002.TA.26-223-j1...|    14:32:00|      14:32:00|            1|
|8575939|148.TA.26-650-j19...|    20:24:00|      20:24:00|            7|
|8591093|98.TA.26-2-A-j19-...|    14:34:00|      14:34:00|           16|
|8590762|518.TA.26-510-j19...|    17:04:00|      17:04:00|           25|
|8591173|2152.TA.26-10-j19...|    15:53:00|      15:53:00|            8|
|8590671|114.TA.26-161-j19...|    17:51:00|      17:51:00|           16|
+-------+--------------------+------------+--------

In [17]:
station_ids = [row['station_id'] for row in stations.collect()]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
stop_ids = spark.sparkContext.broadcast(station_ids)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
filtered = stop_time.where(stop_time.stop_id.isin(stop_ids.value))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
filtered.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o3382.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 4 times, most recent failure: Lost task 0.3 in stage 6.0 (TID 12, iccluster070.iccluster.epfl.ch, executor 8): ExecutorLostFailure (executor 8 exited caused by one of the running tasks) Reason: Container killed by YARN for exceeding memory limits. 2.0 GB of 2 GB physical memory used. Consider boosting spark.yarn.executor.memoryOverhead.
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1602)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1590)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1589)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayB

In [14]:
departure = stop_time.alias('departure')
arrival = stop_time.withColumn('stop_sequence', stop_time.stop_sequence - 1).alias('arrival')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
timetable = (
    departure
    .join(arrival, ['trip_id', 'stop_sequence'])
    .select(
        departure.trip_id,
        departure.stop_sequence,
        departure.stop_id.alias('start_id'),
        departure.departure_time.alias('start_time'),
        arrival.arrival_time.alias('stop_time'),
        arrival.stop_id.alias('stop_id')
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
timetable.write.format("csv").save("/user/gflueck/timetable.csv", mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…